# Mandatory Assignment 2


## Integration af Flygtninge på Arbejdsmarkedet 
Af: Mathias Nikolaisen(dnt357), Oliver Mattthews(jvc975) & Clara Maj Folkmann(djz279)

### Problem 1

In [9]:
#1.a
import pandas as pd

df = pd.read_stata('Integration.dta')
df['const'] = 1
print(df.shape)
print(df.columns)

(4800, 7)
Index(['id', 'mand', 'ankomstaar', 'alder', 'aarslon', 'aar', 'const'], dtype='object')


In [10]:
df_dummies = pd.get_dummies(df, columns=['ankomstaar'], prefix='dummy_ankomstaar')
summary_table1 = df.groupby(['mand', 'aar']).agg(
    mean_aarslon=('aarslon', 'mean'),
    mean_alder=('alder', 'mean'),
    mean_ankomstaar=('ankomstaar', 'mean') 
).reset_index()

print(summary_table1)

   mand     aar  mean_aarslon  mean_alder  mean_ankomstaar
0   0.0  2017.0  31124.437500   28.628283      2014.981812
1   0.0  2018.0  40045.046875   29.628283      2014.981812
2   1.0  2017.0  61324.046875   28.670341      2015.017822
3   1.0  2018.0  90906.500000   29.670341      2015.017822


In [8]:
summary_table2 = df.groupby(['mand', 'aar']).agg(
    median_aarslon=('aarslon', 'median'),
    std_aarslon=('aarslon', 'std'),
    min_aarslon=('aarslon', 'min'),
    max_aarslon=('aarslon', 'max'),
    std_alder=('alder', 'std'),
    min_alder=('alder', 'min'),
    max_alder=('alder', 'max'),
).reset_index()

print(summary_table2)

   mand     aar  median_aarslon   std_aarslon   min_aarslon    max_aarslon  \
0   0.0  2017.0    28953.601562  12640.760314   9058.370117  127671.335938   
1   0.0  2018.0    34427.351562  26195.921317   4526.069824  277230.656250   
2   1.0  2017.0    55599.742188  27148.603614  10105.024414  240001.187500   
3   1.0  2018.0    77553.937500  56593.295207   7315.529297  602016.812500   

   std_alder  min_alder  max_alder  
0   6.963701       18.0       60.0  
1   6.963701       19.0       61.0  
2   7.066349       17.0       65.0  
3   7.066349       18.0       66.0  


In [15]:
#1.b
import statsmodels.api as sm
import numpy as np

df['YSM'] = df['aar'] - df['ankomstaar']
df_2017 = df[df['aar'] == 2017].copy()
y = np.log(df_2017['aarslon'])
X = df_2017[['alder', 'YSM']]
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                aarslon   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     39.84
Date:                Sun, 06 Oct 2024   Prob (F-statistic):           9.53e-18
Time:                        10:30:45   Log-Likelihood:                -1639.8
No. Observations:                2400   AIC:                             3286.
Df Residuals:                    2397   BIC:                             3303.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.4231      0.047    219.799      0.0

In [17]:
#1.c
beta_1 = model.params['alder']
beta_2 = model.params['YSM']

combined_estimate = beta_1 + beta_2

cov_matrix = model.cov_params()

se_combined = np.sqrt(
    cov_matrix.loc['alder', 'alder'] + 
    cov_matrix.loc['YSM', 'YSM'] + 
    2 * cov_matrix.loc['alder', 'YSM']
)

print(f"Combined estimate (beta_1 + beta_2): {combined_estimate}")
print(f"Standard error of the combined estimate: {se_combined}")


Combined estimate (beta_1 + beta_2): 0.09416504673465748
Standard error of the combined estimate: 0.011960398278925283


### Problem 2

In [18]:
#2.a
df_2017_2018 = df[df['aar'].isin([2017, 2018])].copy()

y = np.log(df_2017_2018['aarslon'])

X = df_2017_2018[['alder', 'YSM', 'ankomstaar']]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                aarslon   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     125.4
Date:                Sun, 06 Oct 2024   Prob (F-statistic):           3.61e-78
Time:                        10:50:19   Log-Likelihood:                -4102.0
No. Observations:                4800   AIC:                             8212.
Df Residuals:                    4796   BIC:                             8238.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -457.8689     38.818    -11.795      0.0

### Problem 3

In [7]:
#3.a
model_tp = sm.OLS(df['taxrate'], df[['logpop','const']]).fit()
df['res1'] = model_tp.resid
print(model_tp.summary())

                            OLS Regression Results                            
Dep. Variable:                taxrate   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     3.862
Date:                Wed, 18 Sep 2024   Prob (F-statistic):             0.0523
Time:                        11:46:54   Log-Likelihood:                -127.16
No. Observations:                  98   AIC:                             258.3
Df Residuals:                      96   BIC:                             263.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
logpop        -0.2273      0.116     -1.965      0.0

In [8]:
#3.b
df['logrev'] = y
model_tr = sm.OLS(df['logrev'], df[['res1','const']]).fit()
print(model_tr.summary())

                            OLS Regression Results                            
Dep. Variable:                 logrev   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                   0.06626
Date:                Wed, 18 Sep 2024   Prob (F-statistic):              0.797
Time:                        11:46:54   Log-Likelihood:                -112.50
No. Observations:                  98   AIC:                             229.0
Df Residuals:                      96   BIC:                             234.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
res1           0.0226      0.088      0.257      0.7